In [ ]:
import csv

input_file = "../static/data/가구인테리어_train.csv"
output_file = "../static/data/가구인테리어_train_qa.csv"

qa_pairs = []

with open(input_file, "r", encoding="utf-8-sig") as infile:
    reader = csv.DictReader(infile)
    utterances = [row["발화문"] for row in reader]
    
    for i in range(0, len(utterances), 2):
        if i + 1 < len(utterances):  # 두 줄씩 묶을 수 있는지 확인
            question = utterances[i]      # 첫 번째는 Q
            answer = utterances[i + 1]    # 두 번째는 A
            qa_pairs.append([question, answer])
            
with open(output_file, "w", encoding="utf-8-sig", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["Q", "A"])
    writer.writerows(qa_pairs)

print(f"변환 완료: {output_file}")

In [ ]:
!pip install pandas matplotlib seaborn wordcloud konlpy

In [ ]:
!pip install git+https://github.com/wisenut-research/konoise.git


In [1]:
# 질문/답변 길이 분석
import pandas as pd
df = pd.read_csv('가구인테리어_train_qa.csv')

In [ ]:
print(df.head())

df['Q_length'] = df['Q'].apply(lambda x: len(str(x)))
df['A_length'] = df['A'].apply(lambda x: len(str(x)))

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['Q_length'], kde=True, color='blue')
plt.title('질문 길이 분포')

plt.subplot(1, 2, 2)
sns.histplot(df['A_length'], kde=True, color='green')
plt.title('답변 길이 분포')

plt.tight_layout()
plt.show()

In [ ]:
from konlpy.tag import Okt

okt = Okt()

def extract_nouns(text):
    return okt.nouns(text)

df = df.head(100000)

df['Q_nouns'] = df['Q'].apply(extract_nouns)
df['A_nouns'] = df['A'].apply(extract_nouns)

q_nouns = sum(df['Q_nouns'], [])
a_nouns = sum(df['A_nouns'], [])

q_noun_counts = pd.Series(q_nouns).value_counts()
a_noun_counts = pd.Series(a_nouns).value_counts()

# 자주 등장하는 질문과 답변 시각화
plt.figure(figsize=(12, 6))

# 질문 빈도 시각화
plt.subplot(1, 2, 1)
q_noun_counts.head(10).plot(kind='barh', color='skyblue')
plt.title('Top 10 Most Frequent Nouns in Questions')

# 답변 빈도 시각화
plt.subplot(1, 2, 2)
a_noun_counts.head(10).plot(kind='barh', color='lightgreen')
plt.title('Top 10 Most Frequent Nouns in Answers')

plt.tight_layout()
plt.show()

In [2]:
from wordcloud import WordCloud

# 한글 폰트 경로 지정 (예: 나눔고딕)
font_path = '/font/NanumGothic.ttf'

# 질문과 답변의 텍스트 합치기
text_q = ' '.join(df['Q'].dropna())
text_a = ' '.join(df['A'].dropna())

# Word Cloud 생성 (한글 폰트 설정)
wordcloud_q = WordCloud(font_path=font_path, width=800, height=400, background_color='white').generate(text_q)
wordcloud_a = WordCloud(font_path=font_path, width=800, height=400, background_color='white').generate(text_a)

# Word Cloud 시각화
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.imshow(wordcloud_q, interpolation='bilinear')
plt.title('Word Cloud for Questions')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(wordcloud_a, interpolation='bilinear')
plt.title('Word Cloud for Answers')
plt.axis('off')

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
from noise_generator import *
# 노이즈 추가 알고리즘
# [disattach-letters 문자 분리, prob=1.] 행복한 ㄱㅏ정은 모두ㄱㅏ 닮았ㅈㅣ만, 불행한 ㄱㅏ정은 모두 ㅈㅓㅁㅏㄷㅏ의 ㅇㅣ유로 불행ㅎㅏㄷㅏ.
# [change-vowels 자음 변경, prob=1.] 행복한 갸정은 묘듀갸 닮았지만, 불행한 갸정은 묘듀 져먀댜의 이우료 불행햐댜.
# [palatalization 구개음화, prob=1.] 행보칸 가정은 모두가 닮았지만, 불행한 가정은 모두 저마다의 이유로 불행하다.
def add_noise(text):
    text =  (text, prob=1.)
    return text

# 증강할 데이터 선택(길이가 50 이상인 문장의 랜덤 추출)
min_length = 80
sampled_df = df[(df['Q'].apply(lambda x: len(x) >= min_length))]
#sample_size = int(len(filtered_df)* 0.3)
#sampled_df = filtered_df.sample(n=sample_size, random_state=42).copy()

# Q 데이터 증강 (샘플링된 데이터만)
sampled_df['Q_augmented'] = sampled_df['Q'].apply(add_noise)

# 원본 Q, A와 증강된 Q를 새로운 데이터프레임에 합치기
augmented_df = sampled_df[['Q_augmented', 'A']].rename(columns={'Q_augmented': 'Q'})

# 증강 데이터를 새로운 파일로 저장
augmented_df.to_csv('augmented_data_palatalization3.csv', index=False, encoding='utf-8-sig')

print("증강된 데이터가 'augmented_data.csv'로 저장되었습니다.")

증강된 데이터가 'augmented_data.csv'로 저장되었습니다.


C:\Users\seun0\AppData\Local\Temp\ipykernel_15124\2036932676.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df['Q_augmented'] = sampled_df['Q'].apply(add_noise)
